In [1]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
xm = load_model('transmitter', device=device) # reconstruction
model = load_model('text300M', device=device) # text embedding
diffusion = diffusion_from_config(load_config('diffusion')) # latent diffusion

In [4]:
batch_size = 4
guidance_scale = 15.0
prompt = "a pyramid"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [5]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAODi4uDf4N7d3t3d3t3c3t3c3dzc3drc3dzb3dzb29vb3Nza3N…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAJ6Tep6SeZuReZ2Qd5yQeJyQd5yQdpuQeJmQeJyPd5yPdpyPdZ…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAKuVeaqUeaWRd6KPd6KPdaKOdKKOc6GOdKKNc6GNc6CNc6CNcq…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAK+aea+aeLCZeK+Zea+ZeK+Zd66aeq6ZeK6Zd6+YeK+Yd66YeK…

In [6]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh
import open3d as o3d

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    
    # mesh = o3d.io.read_triangle_mesh(f'example_mesh_{i}.ply')
    # o3d.visualization.draw_geometries([mesh])
    # with open(f'example_mesh_{i}.obj', 'w') as f:
    #     t.write_obj(f)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Bin size was too small in the coarse rasterization phase. This caused an overflow, meaning output may be incomplete. To solve, try increasing max_faces_per_bin / max_points_per_bin, decreasing bin_size, or setting bin_size to 0 to use the naive rasterization.Bin size was too small in the coarse rasterization phase. This caused an overflow, meaning output may be incomplete. To solve, try increasing max_faces_per_bin / max_points_per_bin, decreasing bin_size, or setting bin_size to 0 to use the naive rasterization.Bin size was too small in the coarse rasterization phase. This caused an overflow, meaning output may be incomplete. To solve, try increasing max_faces_per_bin / max_points_per_bin, decreasing bin_size, or setting bin_size to 0 to use the naive rasterization.Bin size was too small in the coarse rasterization pha

In [7]:
# rescale mesh based on volume constraint (NOTE: requires a watertight mesh)
# v = o3d.geometry.TriangleMesh.get_volume(mesh)
# print("\n\nVolume of mesh:", v)

In [8]:
# remove elements of the mesh to satisfy simplicity constraint

# vertex clustering

# connected components

In [10]:
# sample from the mesh to generate point cloud
mesh = o3d.io.read_triangle_mesh('example_mesh_1.ply')
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])
point_cloud = mesh.sample_points_uniformly(2048)
o3d.visualization.draw_geometries([point_cloud])
alternate_point_cloud = mesh.sample_points_poisson_disk(2048)
o3d.visualization.draw_geometries([alternate_point_cloud])